In [16]:
import gym
import collections
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [17]:
# Set Hyperparameters
lr = 0.0005
gamma = 0.98
buffer_limit = 50000
batch_size = 32

In [18]:
"""
Experience Replay
  - Buffer

    - put
      - To store sets of transition
    - sample
"""
class ReplayBuffer():
  def __init__(self):
    self.buffer = collections.deque(maxlen=buffer_limit) # 50,000개 저장

  def put(self, transition):
    self.buffer.append(transition)

  def sample(self, n):
    # n은 mini-batch size
    mini_batch = random.sample(self.buffer, n)
    s_lst, a_lst, r_lst, s_prime, done_mask_lst = [], [], [], [], []

    for transition in mini_batch:
      s, a, r, s_prime, done_mask = transition
      s_lst.append(s)
      a_lst.append([a])
      r_lst.append([r])
      s_prime.append(s_prime)
      done_mask_lst.append([done_mask])

    return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
          torch.tensor(r_lst), torch.tensor(s_prime, dtype=torch.float), \
          torch.tensor(done_mask_lst, dtype=np.bool_)

  def size(self):
    return len(self.buffer)

In [19]:
"""
Model Architecture
  - Qnet
"""
class Qnet(nn.Module):
  def __init__(self):
    super(Qnet, self).__init__()
    self.fc1 = nn.Linear(4, 128)
    self.fc2 = nn.Linear(128,128)
    self.fc3 = nn.Linear(128,2)

  def forward(self,x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

  def sample_action(self, obs, epsilon):
    out = self.forward(obs)
    coin = random.random() # 0~1 사이의 실수값
    if coin < epsilon:
      return random.randint(0,1)
    else:
      return out.argmax().item()

In [20]:
"""
train
"""
def train(q_net, q_target, memory, optimizer):
  for i in range(10):
    s, a, r, s_prime, done_mask = memory.sample(batch_size)

    q_out = q_net(s) # q-network에서 학습
    q_a = q_out.gather(1,a)
    max_q_prime = q_target.max(1)[0].unsqueeze(1) # optimal q val
    target = r + gamma * max_q_prime * done_mask # done_mask: 1 - 진행중 0 - 종료
    loss = F.mse_loss(q_a, target) # mse loss 사용해보자
    # loss = F.smooth_l1_loss(q_a,target) # smooth l1 loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [24]:
"""
main
"""
env = gym.make('CartPole-v1')
q = Qnet()
q_target = Qnet()
q_target.load_state_dict(q.state_dict())
memory = ReplayBuffer()

print_interval = 20
score = 0.0
optimizer = optim.Adam(q.parameters(), lr=lr)

for n_epi in range(10000):
  epsilon = max(0.01, 0.08 - 0.01 * (n_epi/200))
  s= env.reset()
  done = False

  while not done:
    a = q.sample_action(torch.from_numpy(s).float(), epsilon)
    # a = q.sample_action(s, epsilon)
    s_prime, r, done, truncated, info = env.step(a)
    done = bool(done)
    done_mask = 0.0 if done else 1.0 # 진행중이면 1 종료면 0
    memory.put((s,a,r/100.0,s_prime,done_mask))
    s = s_prime

    score += r
    if done:
      break

    if memory.size() > 2000:
      train(q,q_target,memory,optimizer)

    if n_epi % print_interval == 0 and n_epi != 0:
      q_target.load_state_dict(q.state_dict())
      print(f"[n_episode]: {n_epi}, [score]: {score/print_interval}, [epsilon]: {epsilon*100}")

      scroe = 0.0
  env.close()

/usr/local/lib/python3.11/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


AttributeError: module 'numpy' has no attribute 'bool8'